# Lab 08 - Redshift
#### Izzy Valdivia
#### 11/22/2025


### Native Optimized CTAS Table:  
`optimized_orders`  
This represents the data that gets loaded into Redshift that gets queries more frequently
 
______________
  
### External Table:  
`spectrum_raw.orders`  
Not in redshift. This was used to load data into redshift, though. 


#### Goal: 
Analyze the tradeoff between query performance and query flexibility when using an optimized CTAS table versus an external table.

In [2]:
from pyathena import connect